In [1]:
# import other
%run other.ipynb

In [2]:
import numpy as np
import time
import math
from PIL import Image, ImageDraw

In [13]:
class filter_class:
    # Gaussian filter
    '''
    image 為 PIL.Image.Image
    n為陣列值3:3*3  5:5*5 7*7
    n為奇數，為mesh的大小
    '''
    def gaussian_filter(image_array_mesh, image, Variance_mode ='gaussian',n=3):
        import math
        image_array_mesh = image_array_mesh
        
        # Gaussian所需要的參數
        gauss_value = np.zeros(3)
        Variance_1 = np.zeros(3)
        Variance = np.zeros(3)
        average = np.zeros(3)
        num = n**2

        # 將大mesh逐一計算
        count_y = 0
        x_scale, y_scale = other_class.mesh_search(image_array_mesh, image)
        for x in range(x_scale):
            for y in range(y_scale):
                # Gaussian mesh
                mesh_temp = image_array_mesh[y, x]
                mesh = np.zeros((n,n,(3)), dtype='uint8')
                x_mesh, y_mesh, z = mesh_temp.shape #5 5 3

                count_x = 0
                count_y = 0
                #-----------------------------------計算Variance------------------------------------
                # 對一個大mesh做 卷積，逐一取出小 mesh
                for y2 in np.arange(0, y_mesh, 1):
                    for x2 in np.arange(0, x_mesh, 1):
                        if   (x2+n) < x_mesh and (y2+n) < y_mesh:
                            mesh = mesh_temp[x2:x2+n, y2:y2+n]
                            #print(1)
                        elif (x2+n) == x_mesh and (y2+n) < y_mesh:
                            mesh = mesh_temp[x_mesh-n:x_mesh, y2:y2+n]
                            #print(2)
                        elif (x2+n) < x_mesh and (y2+n) == y_mesh:
                            mesh = mesh_temp[x2:x2+n, y_mesh-n:y_mesh]
                            #print(3)
                        elif (x2+n) == x_mesh and (y2+n) == y_mesh:
                            mesh = mesh_temp[x_mesh-n:x_mesh, y_mesh-n:y_mesh]
                            #print(4)
                        else:
                            break
                            print("X")
                # 計算Variance
                if Variance_mode == 'gaussian': 
                    Variance_1 = np.zeros(3)
                    Variance = np.zeros(3)
                    for y2 in np.arange(0, n, 1):
                        for x2 in np.arange(0, n, 1):
                            for rgb in range(3):
                                average[rgb] = np.mean(mesh[:,:,(rgb)])
                                Variance_1[rgb] = (mesh[x2,y2,(rgb)]-average[rgb])**2
                                Variance[rgb] += Variance_1[rgb]
                    for rgb in range(3):
                        Variance[rgb] = Variance[rgb] / num
                        if Variance[rgb] ==0 or Variance[rgb] < 0.001:
                            Variance[rgb] = 0.01
                else:
                    Variance = np.array([Variance_mode, Variance_mode, Variance_mode])
                    
                #------------將各mesh的Variance逐一計算高斯權重並且乘上mesh內的RGB值，取代(0,0)的pixel----------------------------------
                for y2 in np.arange(0, y_mesh, 1):
                    for x2 in np.arange(0, x_mesh, 1):
                        if   (x2+n) < x_mesh and (y2+n) < y_mesh:
                            mesh = mesh_temp[x2:x2+n, y2:y2+n]
                            #print(1)
                        elif (x2+n) == x_mesh and (y2+n) < y_mesh:
                            mesh = mesh_temp[x_mesh-n:x_mesh, y2:y2+n]
                            #print(2)
                        elif (x2+n) < x_mesh and (y2+n) == y_mesh:
                            mesh = mesh_temp[x2:x2+n, y_mesh-n:y_mesh]
                            #print(3)
                        elif (x2+n) == x_mesh and (y2+n) == y_mesh:
                            mesh = mesh_temp[x_mesh-n:x_mesh, y_mesh-n:y_mesh]
                            #print(4)
                        else:
                            break
                            print("X")

                        gaussian_mesh = np.zeros((n,n,(3)), dtype='float')
                        count = 0
                        for cy in range(n):
                            for cx in range(n):
                                for y1 in np.arange(-cy, n-cy, 1):
                                    for x1 in np.arange(-cx, n-cx, 1):
                                        for rgb in range(3):
                                            #print(Variance[rgb])
                                            #print(x1)
                                            #print(y1)
                                            gaussian_mesh[x1, y1, (rgb)] = (1/(2*(math.pi)*Variance[rgb]))*(math.exp(-(x1**2+y1**2)/(2*Variance[rgb])))
                                            #print(gaussian_mesh[x1, y1, (rgb)])
                                #print(gaussian_mesh)
                                for rgb in range(3):
                                    sum_gaussian = np.sum(gaussian_mesh[:,:,(rgb)])
                                    if sum_gaussian < 0.000001:
                                        sum_gaussian = 0.01
                                    gaussian_mesh[:,:,(rgb)] = gaussian_mesh[:,:,(rgb)] / sum_gaussian
                                    #print(gaussian_mesh[:,:,(rgb)])
                                    mesh[cx, cy,(rgb)] = np.sum(mesh[:,:,(rgb)]*gaussian_mesh[:,:, (rgb)])
                                    #print(a[rgb])
        return image_array_mesh
    
    def Sharpening_filter_3x3(image_array, n=3):
        '''
        這裡的卷積方法是沿著x方向
        n為kernel
        這裡會使用邊緣擴增的方式來完成捲積
        '''
        # image.size   (233, 204)
        h, w, r = image_array.shape # (204, 233, 3)
        
        zero_array = np.zeros(1)
        image_array = np.insert(image_array, 0, zero_array, axis=0) #上
        image_array = np.insert(image_array, h, zero_array, axis=0) #下
        image_array = np.insert(image_array, 0, zero_array, axis=1) #左
        image_array = np.insert(image_array, w, zero_array, axis=1) #右
        
        h1, w1, r1 = image_array.shape
        
        # 銳化的mesh
        window = ([
            [-1, -1, -1],
            [-1,  8, -1],
            [-1, -1, -1]
        ])
        sharpen_mesh_list = []
        for y in np.arange(0, h1, 1):
            for x in np.arange(0, w1, 1):
                if   (x+n > w1) or (y+n > h1):
                    continue
                else:
                    mesh = image_array[y:y+n, x:x+n]
                    #print(mesh.shape)
                    #print(y, x)
                for rgb in range(3):
                    pixel_value =  np.sum(mesh[rgb]*window)/4
                    if pixel_value<0:
                        pixel_value = 0
                    elif pixel_value > 255:
                        pixel_value = 255
                    else:
                        pass
                    sharpen_mesh_list.append(pixel_value)
        # 將list轉成array
        sharpen_mesh_array = np.array(sharpen_mesh_list, dtype='uint8')
        #對array做reshape
        sharpen_mesh_array_reshape = sharpen_mesh_array.reshape([h, w ,(r)])
        
        return sharpen_mesh_array_reshape
    
    '''
    # y方向的卷積
    def Sharpening_filter_laplacian_3x3(image_array):
        h, w, r = image_array.shape
        zero_array = np.zeros(n)
        image_array = np.insert(image_array, 0, zero_array, axis=0) #上
        image_array = np.insert(image_array, h, zero_array, axis=0) #下
        image_array = np.insert(image_array, 0, zero_array, axis=1) #左
        image_array = np.insert(image_array, w, zero_array, axis=1) #右
        
        h1, w1, r1 = image_array.shape
        
        # 銳化的mesh
        window = ([
            [-1, -1, -1],
            [-1,  8, -1],
            [-1, -1, -1]
        ])
        init = 0
        temp_b = np.zeros([h,1,(3)], dtype='uint8')
        sharpen_mesh_list = []
        for x in np.arange(0, w1-1, 1):
            if init == 1:

                a = np.array(sharpen_mesh_list , dtype='uint8')
                #print(a.shape)
                b = a.reshape([h,1,(3)])
                temp_b = np.concatenate((temp_b, b), axis = 1)
                sharpen_mesh_list = []
            for y in np.arange(0, h1, 1):
                if x==w1:
                    continue
                init = 1
                if   (x+n > w1) or (y+n > h1):
                    continue
                else:
                    mesh = image_array[y:y+n, x:x+n]
                    #print(mesh.shape)
                    #print(y, x)
                for rgb in range(3):
                    pixel_value =  np.sum(mesh[rgb]*window)/8
                    if pixel_value<0:
                        pixel_value = 0
                    elif pixel_value>255:
                        pixel_value = 255
                    else:
                        pass
                    sharpen_mesh_list.append(pixel_value)
        temp_b = np.delete(temp_b, 0, axis = 1)
        return temp_b
    '''
    
    def Sharpening_filter_laplacian_5x5(image_array, n=5):
        h, w, r = image_array.shape

        zero_array = np.zeros(1)
        image_array = np.insert(image_array, 0, zero_array, axis=0) #上
        image_array = np.insert(image_array, h, zero_array, axis=0) #下
        image_array = np.insert(image_array, 0, zero_array, axis=0) #上
        image_array = np.insert(image_array, h, zero_array, axis=0) #下

        image_array = np.insert(image_array, 0, zero_array, axis=1) #左
        image_array = np.insert(image_array, w, zero_array, axis=1) #右
        image_array = np.insert(image_array, 0, zero_array, axis=1) #左
        image_array = np.insert(image_array, w, zero_array, axis=1) #右
        h1, w1, r1 = image_array.shape
        # 銳化的mesh 5*5
        window = ([
            [-1, -1, -1, -1, -1],
            [-1,  2,  2,  2, -1],
            [-1,  2,  8,  2, -1],
            [-1,  2,  2,  2, -1],
            [-1, -1, -1, -1, -1]
        ])
        sharpen_mesh_list = []
        for y in np.arange(0, h1, 1):
            for x in np.arange(0, w1, 1):
                if   (x+n > w1) or (y+n > h1):
                    continue
                else:
                    mesh = image_array[y:y+n, x:x+n]
                    #print(mesh[:,:,(0)])
                    #print(y, x)
                for rgb in range(3):
                    pixel_value =  np.sum(mesh[:,:,(rgb)]*window)/8
                    if pixel_value < 0:
                        pixel_value = 0
                    elif pixel_value>255:
                        pixel_value = 255
                    else:
                        pass
                    sharpen_mesh_list.append(pixel_value)
        sharpen_mesh_array = np.array(sharpen_mesh_list, dtype='uint8')
        sharpen_mesh_array_reshape = sharpen_mesh_array.reshape([h, w ,(r)])
        return sharpen_mesh_array_reshape
    
    def median_filter(image_array, n = 3):
        '''
        中值濾波
        x方向卷積
        n最小為3 (限奇數)
        '''
        h, w, r = image_array.shape
        zero_array = np.zeros(1)
        padding = int(n//2)
        for i in range(padding):
            image_array = np.insert(image_array, 0, zero_array, axis=0) #上
            image_array = np.insert(image_array, h, zero_array, axis=0) #下
            image_array = np.insert(image_array, 0, zero_array, axis=1) #左
            image_array = np.insert(image_array, w, zero_array, axis=1) #右
        
        h1, w1, r1 = image_array.shape
        median_list = []
        median_value = 0
        for y in np.arange(0, h1, 1):
            for x in np.arange(0, w1, 1):
                if   (x+n > w1) or (y+n > h1):
                    continue
                else:
                    mesh = image_array[y:y+n, x:x+n]
                    #print(mesh.shape)
                    #print(y, x)
                for rgb in range(3):
                    median_value = np.median(mesh[:,:,(rgb)])
                    median_list.append(median_value)
                    
        sharpen_mesh_array = np.array(median_list, dtype='uint8')
        sharpen_mesh_array_reshape = sharpen_mesh_array.reshape([h, w ,(r)])
        return sharpen_mesh_array_reshape
    
    def average_filter(image_array, n = 3):
        '''
        中值濾波
        x方向卷積
        n最小為3 (限奇數)
        '''
        h, w, r= image_array.shape
        zero_array = np.zeros(1)
        padding = int(n//2)
        for i in range(padding):
            image_array = np.insert(image_array, 0, zero_array, axis=0) #上
            image_array = np.insert(image_array, h, zero_array, axis=0) #下
            image_array = np.insert(image_array, 0, zero_array, axis=1) #左
            image_array = np.insert(image_array, w, zero_array, axis=1) #右
        
        h1, w1, r1 = image_array.shape
        average_list = []
        average_value = 0
        for y in np.arange(0, h1, 1):
            for x in np.arange(0, w1, 1):
                if   (x+n > w1) or (y+n > h1):
                    continue
                else:
                    mesh = image_array[y:y+n, x:x+n]
                    #print(mesh.shape)
                    #print(y, x)
                for rgb in range(3):
                    average_value = np.mean(mesh[:,:,(rgb)])
                    average_list.append(average_value)
                    
        sharpen_mesh_array = np.array(average_list, dtype='uint8')
        sharpen_mesh_array_reshape = sharpen_mesh_array.reshape([h, w ,(r)])
        return sharpen_mesh_array_reshape

In [ ]:
#    def median_filter(image_array_mesh, image, ):

In [4]:
np.zeros(3)

array([0., 0., 0.])